# The Viewer Component
The interactive viewer for IFC models has been extended from Thomas Paviots' excellent [JupyterRenderer](https://github.com/tpaviot/pythonocc-core/blob/master/src/Display/WebGl/jupyter_renderer.py) and offeres a number of functionalities:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import collections
import math
import uuid
import random
from itertools import islice

import ifcopenshell
import open3d as o3d
import numpy as np
import pymeshlab as ml
from tqdm import tqdm
from ifcopenshell.util.selector import Selector
from ipywidgets import interact
from compas.geometry import oriented_bounding_box_numpy
from scipy.spatial import distance

from OCC.Core.Bnd import Bnd_Box, Bnd_OBB
from OCC.Core.BRepBndLib import brepbndlib_AddOBB
from OCC.Core.BRepPrimAPI import (BRepPrimAPI_MakeBox, 
BRepPrimAPI_MakeSphere, BRepPrimAPI_MakeCylinder)
from OCC.Core.gp import gp_Pnt, gp_XYZ, gp_Ax2, gp_Dir

from utils.JupyterIFCRenderer import JupyterIFCRenderer

In [ ]:
m = ifcopenshell.open("../merged_full.ifc")
system_dict_file = "../WestDeckBox.nwd_aggregation.json"
#m = ifcopenshell.open("data/231110AC-11-Smiley-West-04-07-2007.ifc")k

## Relationship Identification

###  aggrgegation relationships

In [ ]:
# flatten info dictionary
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten(el, new_key_2, sep=sep))
                else:
                    
                    items.append(el)
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten(v, new_key, sep=sep))
        else:
            print(type(v))
            items.append(v)
    return (items)


def get_systems(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        out_dict[sys_name] = flatten(sys)
        print(sys_name, len(out_dict[sys_name]))
    return out_dict
        

get_systems(system_dict_file)    
    
    

### Topological relationships

In [ ]:
# flatten info dictionary to branch level
def flatten_branch(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten_branch(el, new_key_2, sep=sep))
                else:
                    
                    items.append((k, el))
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten_branch(v, new_key, sep=sep))
        else:
            print(type(v))
            items.append((k,v))
    return (items)


# get components of each branch
def get_branches(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        items = flatten_branch(sys)
        print(items)
        d = {}
        for branch, e in items:
            if not branch in d:
                d[branch] = []
            d[branch] += [e]
        out_dict[sys_name] = d
        
        #out_dict[sys_name] = flatten_branch(sys)
        print(sys_name, len(out_dict[sys_name]))
    return out_dict
        
system = 'West-DeckBox-Pipe.rvm'
branches = get_branches(system_dict_file)[system]    


In [ ]:


# # draw links between connected elements
# def visualize_branches(branches, ifc, contiguous=False):
#     vis_dict = {}
#     for k, val in branches.items():
#         vis_elements = []
#         for element in val:
#             if element in pipe_names:
#                 vis_elements.append((element, pipe_type))
#             elif element in fitting_names:
#                 vis_elements.append((element, fitting_type))
#         vis_dict[k] = vis_elements

#     error_count = 0
#     # enumerate through branches
#     count = 0
#     for k, val in tqdm(vis_dict.items()):
#         if count == 50:
#             break
#         branch_size = len(val)
#         for i, element in enumerate(val):
#             #check if element is not the last element
#             if (i+1) < branch_size:
#                 try:
#                     draw_relationship(element[0], element[1], 
#                           val[i+1][0], val[i+1][1], ifc)
#                 except:
#                     error_count +=1
#         count +=1

#     print(error_count)
#     #print(vis_dict)
    
def visualize_branches(branches, ifc, contiguous=True, dist_thresh=0.002):
    pipe_type = 'IFCPIPESEGMENT'
    fitting_type = 'IFCPIPEFITTING'

    pipe_selector = Selector()
    fitting_selector = Selector()
    pipes = pipe_selector.parse(ifc, '.' + pipe_type)
    fittings = fitting_selector.parse(ifc, '.' + fitting_type)
    fitting_names = [f.Name for f in fittings]
    pipe_names = [p.Name for p in pipes]
    print(pipes[0].Name)

    vis_dict = {}
    for k, val in branches.items():
        vis_elements = []
        connect = True
        for element in val:
            if element in pipe_names:
                vis_elements.append((element, pipe_type, connect))
                connect = True
            elif element in fitting_names:
                vis_elements.append((element, fitting_type, connect))
                connect = True
            else:
                connect = False
        vis_dict[k] = vis_elements

    error_count = 0
    # enumerate through branches
    count = 0
    for k, val in tqdm(vis_dict.items()):
        if count == 10:
            break
        branch_size = len(val)
        for i, element in enumerate(val):
            #check if element is not the last element
            if (i+1) < branch_size:
                try:
                    if val[i+1][2] or not contiguous:
                        draw_relationship(element[0], element[1], 
                              val[i+1][0], val[i+1][1], ifc)
                    else:
                        if element_distance(element[0], element[1], 
                              val[i+1][0], val[i+1][1], ifc) < dist_thresh:
                            draw_relationship(element[0], element[1], 
                              val[i+1][0], val[i+1][1], ifc)
                except:
                    error_count +=1
        count +=1

    print(error_count)

    
visualize_branches(branches, m)

## Visualization

In [ ]:
viewer = JupyterIFCRenderer(m, size=(400,300))
viewer.setAllTransparent()
viewer

### aggregation relationships

In [ ]:
picker = viewer.colorPicker()
picker


In [ ]:
out_dict = {'HVAC':['Rohrtypen:Kupfer - Hartgelötet:7718868', 'Rohrtypen:Kupfer - Hartgelötet:7718886', ],
           'electrical':[ 'Rohrtypen:Kupfer - Hartgelötet:7718872', 'Rohrtypen:Kupfer - Hartgelötet:7718880']}

# PAINT A SET OF ELEMENTS IN ONE COLOUR
def systemSelect(system):
    selector = Selector()
    for e in out_dict[system]:
        element = selector.parse(m, '.IfcProduct[Name *= "' + e + '"]')[0]
        viewer.setColor(element, picker.value)
    return system

interact(systemSelect, system=['HVAC', 'electrical'])

Instances of building elements with represenations can be selected interactivly. Information such as the attributes `GUID`, `Name` etc. are displayed to the left of the 3D viewport.

In [ ]:
# reset colours
viewer.setDefaultColors()

### topological relationships



## replace jupyter renderer

1. Compute the bounding box of ifc product directly from points
2. generate ifc elements to indicate relationships


In [ ]:
element_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element_type = "IFCPIPESEGMENT"

selector = Selector()
element = selector.parse(
    m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

shape = element.Representation.Representations[0].Items[0]
element_coords = np.array(shape.Coordinates.CoordList)
#print(element_coords)
bbox = oriented_bounding_box_numpy(element_coords)
print(bbox)

In [ ]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = m.by_type("IfcOwnerHistory")[0]
project = m.by_type("IfcProject")[0]
context = m.by_type("IfcGeometricRepresentationContext")[0]
floor = m.by_type("IfcBuildingStorey")[0]

# get bounding box of ifc element
def get_oriented_bbox(element_name, element_type):
    # get element and bbox
    selector = Selector()
    element = selector.parse(
        m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

    #print(element.Representation.Representations)
    shape = element.Representation.Representations[0].Items[0]
    element_coords = np.array(shape.Coordinates.CoordList)
    #print(element_coords)
    bbox = oriented_bounding_box_numpy(element_coords)
    
    # identify box orientation
    l1 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[1][0], bbox[1][1], bbox[1][2]))
    l2 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[3][0], bbox[3][1], bbox[3][2]))
    l3 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[4][0], bbox[4][1], bbox[4][2]))
    half_lengths = [l1/2, l2/2, l3/2]
    
    dominant_axis = half_lengths.index(max(half_lengths))
    if dominant_axis == 0:
        dominant_direction = vector_norm([bbox[0][i] - bbox[1][i] 
                                          for i in range(3)])
    elif dominant_axis == 1:
        dominant_direction = vector_norm([bbox[0][i] - bbox[3][i] 
                                          for i in range(3)])
    else:
        dominant_direction = vector_norm([bbox[0][i] - bbox[4][i] 
                                          for i in range(3)])

    dominance_ratio = max(half_lengths)/sorted(half_lengths)[-2]
    center = [(bbox[0][i] + bbox[6][i])/2 for i in range(3)]

    #print(dominance_ratio, dominant_direction)
    #print(element_name, half_lengths, dominant_direction, center)

    #print(center)
    return([dominant_direction, max(half_lengths), 
           half_lengths, center])


def CreateBeam(ifcFile, container, name, section, L, position, 
               direction, owner_history, context):
    Z = 0.,0.,1.
    #print('length', L)
    B1 = ifcFile.createIfcBeam(create_guid(),owner_history , name)
    B1.ObjectType ='beam'
    
    #print(type(position[0]))
    B1_Point =ifcFile.createIfcCartesianPoint ( tuple(position) ) 
    #B1_Point =ifcFile.createIfcCartesianPoint ( (0.0,0.0,0.0) ) 
    B1_Axis2Placement = ifcFile.createIfcAxis2Placement3D(B1_Point)
    B1_Axis2Placement.Axis = ifcFile.createIfcDirection(direction)
    B1_Axis2Placement.RefDirection =ifcFile.createIfcDirection(
        np.cross(direction, Z).tolist())

    B1_Placement = ifcFile.createIfcLocalPlacement(
        container.ObjectPlacement,B1_Axis2Placement)
    B1.ObjectPlacement=B1_Placement
    B1_ExtrudePlacement = ifcFile.createIfcAxis2Placement3D(ifcFile.createIfcCartesianPoint ( (0.,0.,0.) )   )
   
    B1_Extruded=ifcFile.createIfcExtrudedAreaSolid()
    B1_Extruded.SweptArea=section
    B1_Extruded.Position=B1_ExtrudePlacement
    B1_Extruded.ExtrudedDirection = ifcFile.createIfcDirection(Z)
    B1_Extruded.Depth = L

    B1_Repr=ifcFile.createIfcShapeRepresentation()
    B1_Repr.ContextOfItems=context
    B1_Repr.RepresentationIdentifier = 'Body'
    B1_Repr.RepresentationType = 'SweptSolid'
    B1_Repr.Items = [B1_Extruded]
    
    B1_DefShape=ifcFile.createIfcProductDefinitionShape()
    B1_DefShape.Representations=[B1_Repr]
    B1.Representation=B1_DefShape
    
    Flr1_Container = ifcFile.createIfcRelContainedInSpatialStructure(create_guid(),owner_history)
    Flr1_Container.RelatedElements=[B1]
    Flr1_Container.RelatingStructure= container
    
    
def Circle_Section(r, ifcfile):
    B1_Axis2Placement2D =ifcfile.createIfcAxis2Placement2D( 
                          ifcfile.createIfcCartesianPoint( (0.,0.,0.) ) )
    B1_AreaProfile = ifcfile.createIfcCircleProfileDef("AREA")
    B1_AreaProfile.Position = B1_Axis2Placement2D 
    B1_AreaProfile.Radius = r
    return B1_AreaProfile


#get_oriented_bbox(element_name, element_type)

In [ ]:

def get_point_along_axis(init_point, axis, half_length, edge_distance):
    #print(half_length)
    return (init_point + axis*(half_length - edge_distance))


def sq_distance(x1, y1, z1, x2, y2, z2):
    return ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)


def vector_norm(vec):
    den = math.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    return [vec[0]/den, vec[1]/den, vec[2]/den]


# identify a candidate for an edge of the relationship visualization
def get_corner(obb, center_other, edge_distance):
    direction = obb[0]
    center = obb[3]

    candidate1 = [get_point_along_axis(center[i], direction[i], 
                                    obb[1], edge_distance) 
               for i in range(3)]
    candidate2 = [get_point_along_axis(center[i], direction[i], 
                                    -1*obb[1], -1*edge_distance) 
               for i in range(3)]
    # print(corner1, corner2)
    
    dist1 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate1[0],
                   candidate1[1], candidate1[2])    
    dist2 = sq_distance(center_other[0], center_other[1],
                   center_other[2], candidate2[0],
                   candidate2[1], candidate2[2]) 
    
    if dist1 < dist2:
        return candidate1
    else:
        return candidate2

    
# get points belonging to an ifc element
def get_points(element_name, element_type, ifc):
    selector = Selector()
    element = selector.parse(
        ifc, '.' + element_type + '[Name *= "' + element_name + '"]')[0]
    shape = element.Representation.Representations[0].Items[0]
    return (np.array(shape.Coordinates.CoordList))   
    
    
# calculate min distance between two ifc elements
def element_distance(element1_name, element1_type, element2_name, element2_type, ifc):
    points1 = get_points(element1_name, element1_type, ifc)
    points2 = get_points(element2_name, element2_type, ifc)
    return np.min(distance.cdist(points1, points2, 'sqeuclidean'))


e1 = 'ELBOW 2 of BRANCH /AU-8110202-WD-MDA-02/B1'
e1t = 'IFCPIPEFITTING'
e2 = 'TUBE 1 of BRANCH /AU-8110202-WD-MDA-02/B1'
e2t = 'IFCPIPESEGMENT'
element_distance(e1, e1t, e2, e2t, m)
    
def draw_sphere(point, radius, colour, viewer):
    point = gp_Pnt(point)
    ball = BRepPrimAPI_MakeSphere(point, radius).Shape()
    viewer.DisplayShape(ball, shape_color = colour, 
                        transparency=True, opacity=0.8)


def draw_cylinder(p1, p2, radius, colour,  element_name1, element_name2, ifc):
    sectionC1 = Circle_Section(r=radius, ifcfile=ifc)
    name='rel '+element_name1 + ' x ' + element_name2
    ConnectingBeam_1 = CreateBeam(ifc, container=floor, name=name, 
                                  section= sectionC1, 
                                  L=math.sqrt(sq_distance(p1[0],p1[1],p1[2],p2[0],p2[1],p2[2])),
                                  position=([p2[0].item(), p2[1].item(), p2[2].item()]),
                                  direction=((p1[0] - p2[0]).item(), (p1[1] - p2[1]).item(), 
                                             (p1[2] - p2[2]).item()),
                                  owner_history=owner_history, context=context)
#     ifc.create_entity('IfcPipeSegment', GlobalId=ifcopenshell.guid.new(), 
#                       Name='rel '+element_name1 + ' x ' + element_name2)
    
# def draw_cylinder(p1, p2, radius, colour, viewer):
#     x = (p1.X() + p2.X())/2
#     y = (p1.Y() + p2.Y())/2
#     z = (p1.Z() + p2.Z())/2
#     p = gp_Pnt(p2.X(), p2.Y(), p2.Z())
#     v = gp_Dir((p1.X() - p2.X()), (p1.Y() - p2.Y()), (p1.Z() - p2.Z()))
#     ax = gp_Ax2(p, v)
#     h = math.sqrt(sq_distance(p1.X(), p1.Y(), p2.Z(), 
#                               p2.X(), p2.Y(), p2.Z()))
#     print(x,y,z,p,v,ax,h)
    
#     cylinder = BRepPrimAPI_MakeCylinder(ax, radius, h).Shape()
#     viewer.DisplayShape(cylinder, shape_color = colour, 
#                         transparency=True, opacity=0.8)
    

# draw a visual indication of a topological relationship    
def draw_relationship (element_name1, element_type1, 
                       element_name2, element_type2, ifc):
    # define params
    #radius = 0.03
    radius_expansion = 1.3
    colour = 'blue'
    threshold = 0.1
    edge_distance = 0.1
    
    # get bboxes
    obb1 = get_oriented_bbox(element_name1, element_type1)
    obb2 = get_oriented_bbox(element_name2, element_type2)
    
    # if bboxes are roughly square, then use centerpoint, 
    # otherwise use a point closer to the edge
    if obb1[1] < threshold:
        corner1 = obb1[3]
    else:
        corner1 = get_corner(obb1, obb2[3], edge_distance)
    if  obb2[1] < threshold:
        corner2 = obb2[3]
    else:
        corner2 = get_corner(obb2, obb1[3], edge_distance)
    #print(corner1.X(),corner1.Y(),corner1.Z(),corner2.X(),corner2.Y(),corner1.Z())
    #print('rel', corner1, corner2, element_name1, element_name2)
    
    #dynamically scale radius
    radius = max(min(obb1[2]), min(obb2[2])) * radius_expansion
    
    draw_cylinder(corner1, corner2, radius, 'green', 
                  element_name1, element_name2, ifc)
#     draw_sphere(corner1, radius, colour, viewer)
#     draw_sphere(corner2, radius, 'red', viewer)
 

In [ ]:
element1_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element1_type = "IFCPIPESEGMENT"
element2_name = "ELBOW 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
element2_type = "IFCPIPEFITTING"

element3_name = "TUBE 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
element3_type = "IFCPIPESEGMENT"
element4_name = "ELBOW 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
element4_type = "IFCPIPEFITTING"
element5_name = "TUBE 3 of BRANCH /AM-8120227-WD-MDA-01/B1"
element5_type = "IFCPIPESEGMENT"

draw_relationship(element1_name, element1_type, 
                  element2_name, element2_type, m)
draw_relationship(element2_name, element2_type, 
                  element3_name, element3_type, m)
draw_relationship(element3_name, element3_type, 
                  element4_name, element4_type, m)
draw_relationship(element4_name, element4_type, 
                  element5_name, element5_type, m)
#element1_center, element1_coords = get_element_deets()



# centerpoint =gp_Pnt(element1_center)
# ball = BRepPrimAPI_MakeSphere(centerpoint, 0.02).Shape()

In [ ]:
m.write('../edited.ifc')

# Misc

In [ ]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)
pdct_shape = ifcopenshell.geom.create_shape(settings, inst=element)
viewer.DisplayShape(ball, shape_color = 'blue', transparency=False, opacity=0.5)

In [ ]:
o_bbox1 = get_oriented_boundingbox(list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element1)])
bbox_center1 = [o_bbox[0].X(), o_bbox[0].Y(), o_bbox[0].Z()]


In [ ]:
viewer.shapedict.values()

In [ ]:
list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]

In [ ]:
viewer._shapes

In [ ]:
BoundingBox([[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]]).xmin

In [ ]:
[list(viewer.shapedict.keys())[list(viewer.shapedict.values()).index(element)]]

In [ ]:
print(len(element.Representation.Representations[0].Items[0].CoordIndex))

In [ ]:
element.Representation.Representations[0].Items[0].CoordIndex = element.Representation.Representations[0].Items[0].CoordIndex[:1]

## Selecting instances 

The instance object currently selected in the 3D view can be assigned to a avariable using the `.getSelectedProduct()` method. 

Provding the plain varialbe name call sthe `Display()`-method of the instance and prints the line of the original ifc-Datei the  [SPFF-format](https://en.wikipedia.org/wiki/ISO_10303-21).


In [ ]:
selection = viewer.getSelectedProduct()
selection

## Show / Hide Objects
Programatically or interactively selected projects can be hidden or shown in the viewer using `setVisible()`.


In [ ]:
viewer.setVisible(selection, False)
viewer

Opening elements are displayed by default. Let's hide them.

In [ ]:
for opening in m.by_type("IfcOpeningElement"):
    viewer.setVisible(opening, False)

## Coloring Objects
Objects can be colored either by hexadicimal numbers provided as strings as they are common in e.g. in HTML and CSS and can be retrieved from many ressources

In [ ]:

viewer.setColorSelected("#ddffaa")
viewer

If you do not happen to have a color in mind  just call a color picker from the viewer:

In [ ]:
picker = viewer.colorPicker()
picker

In [ ]:
viewer.setColorSelected(picker.value)

## IFC to cloud

sample points from ifc model

In [ ]:
ifc = ifcopenshell.open("../merged_full.ifc")

In [ ]:
element_type = 'IFCPIPESEGMENT'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(len(tees))

In [ ]:
# pick a point uniformly in a triangle
def uniform_triangle(u, v):
    while True:
        s = random.random()
        t = random.random()
        in_triangle = s + t <= 1
        p = s * u + t * v if in_triangle else (1 - s) * u + (1 - t) * v
        yield p


# uniformly sample points within triangle
def sample_points(p, q, r, n):   
    it = uniform_triangle((q - p), (r - p))
    points = np.array(list(islice(it, 0, n)))
    points += p
    return points


# convert an element to a point cloud
def element_to_cloud(element, save_path, density=0):
    shape = element.Representation.Representations[0].Items[0]
    boundaries = np.array(shape.Coordinates.CoordList)
    
    # determine sampling target 
    point_count = len(boundaries)
    samples = 10 if density == 0 else math.ceil(density * 6 / point_count)
        
    # get additional points by sampling from mesh triangles
    limit = point_count -2
    #print (limit)
    centroids = []
    for j in range(0, point_count, 3):
        if j < limit:
            # centroids.append([(boundaries[j][k] + boundaries[j+1][k] 
            # + boundaries[j+2][k])/3 for k in range(3)])
            centroids.extend(sample_points((boundaries[j]), (boundaries[j+1]),
                                           (boundaries[j+2]), samples))
    boundaries = np.concatenate([boundaries, np.array(centroids)])
    #print(len(boundaries))
    
    # downsample to fixed length
    if density > 0:
        boundaries = boundaries[np.random.choice(boundaries.shape[0], density, 
                                                 replace=False), :]
    
    # convert to pointcloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(boundaries)
    o3d.io.write_point_cloud(save_path, pcd)
    return (len(boundaries))

In [ ]:
avg_point_count = 0
for i, element in tqdm(enumerate(tees)):
    save_path = "../pipe_clouds/" + str(i) + ".ply"
    avg_point_count += element_to_cloud(element, save_path, 1000)

avg_point_count = avg_point_count/i
print (avg_point_count)

### Downsampling



In [ ]:
ifc = ifcopenshell.open("../deckboxtee.ifc")

element_type = 'IFCPIPEFITTING'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(tees[0])

In [ ]:
for element in tqdm(tees):
    # create pymeshlab mesh
    #print(element)
    shape = element.Representation.Representations[0].Items[0]
    element_faces = [np.array([i[0]-1, i[1]-1, i[2]-1]) for i in shape.CoordIndex]
    element_coords = np.array(shape.Coordinates.CoordList)
    print(len(element_faces))
#     mesh = ml.Mesh(element_coords, element_faces)
#     ms = ml.MeshSet()
#     ms.add_mesh(mesh, "x")
#     #ms.save_current_mesh("../output.ply")
    
#     # downsample and reassign
#     ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(2))
#     m = ms.current_mesh()
#     shape.CoordIndex = m.face_matrix().tolist()
#     shape.Coordinates.CoordList = m.vertex_matrix().tolist()
    

In [ ]:
ifc.write('../added.ifc') 

In [ ]:
viewer = JupyterIFCRenderer(ifc, size=(400,300))
viewer

In [ ]:
## meshlab downsampling -DOESNT WORK

In [ ]:

print('input mesh has', mesh.vertex_number(), 'vertex and', mesh.face_number(), 'faces')


#Estimate number of faces to have 100+10000 vertex using Euler

#Simplify the mesh. Only first simplification will be agressive
#ms.apply_filter('simplification_quadric_edge_collapse_decimation', targetfacenum=numFaces, preservenormal=False)
ms.apply_filter('simplification_clustering_decimation', threshold=ml.Percentage(5))
print("Decimated to", numFaces, "faces mesh has", ms.current_mesh().vertex_number(), "vertex")


m = ms.current_mesh()
print('output mesh has', m.vertex_number(), 'vertex and', m.face_number(), 'faces')
ms.save_current_mesh('../output.ply')

In [ ]:
print(m.face_matrix())

In [ ]:
shape.CoordIndex = m.face_matrix().tolist()
shape.Coordinates.CoordList = m.vertex_matrix().tolist()